# Predicting Rewards with the State Value Function

The state-falue function is a view of the expected return with respect to each state. Below is the equation, where $G$ is the return, $s$ is the state, $\gamma$ is the discount factor, and $r$ is the reward.

$$ V_{\pi}(s) \doteq \mathbb{E}_{\pi}[ G \vert s] = \mathbb{E}_{\pi}\bigg[ \sum^{T}_{k=0} \gamma^k r_{k} \vert s \bigg] $$

You could estimate the expectation in a few ways, but the simplest is to simply average over al of the observed rewards. To investigate how this equation works, you can perform the calculation on a simple environment that is easy to validate.

## The Environment: A Simple Grid World

To demonstrate, let me use a rediculously simple grid-based environment. This consists of 5 squares, with a cliff on the left-hand side and a goal position on the right. Both are terminating states.

In [1]:
starting_position = 1 # The starting position
cliff_position = 0 # The cliff position
end_position = 5 # The terminating state position
reward_goal_state = 5 # Reward for reaching goal
reward_cliff = 0 # Reward for falling off cliff

def reward(current_position) -> int:
    if current_position <= cliff_position:
        return reward_cliff
    if current_position >= end_position:
        return reward_goal_state
    return 0

def is_terminating(current_position) -> bool:
    if current_position <= cliff_position:
        return True
    if current_position >= end_position:
        return True
    return False

## The Agent

In this simple environment, let us define an agent with a simple random strategy. On every step, the agent randomly decides to go left or right.

In [2]:
def strategy() -> int:
    if np.random.random() >= 0.5:
        return 1 # Right
    else:
        return -1 # Left

In [4]:
import numpy as np
np.random.seed(42)

# Global buffers to perform averaging later
value_sum = np.zeros(end_position + 1)
n_hits = np.zeros(end_position + 1)

n_iter = 10001
for i in range(n_iter):
    position_history = [] # A log of positions in this episode
    current_position = starting_position # Reset
    while True:
        # Append position to log
        position_history.append(current_position)

        if is_terminating(current_position):
            break
        
        # Update current position according to strategy
        current_position += strategy()

    # Now the episode has finished, what was the reward?
    current_reward = reward(current_position)
    
    # Now add the reward to the buffers that allow you to calculate the average
    for pos in position_history:
        value_sum[pos] += current_reward
        n_hits[pos] += 1
        
    # Now calculate the average for this episode and print
    expected_return = ', '.join(f'{q:.2f}' for q in value_sum / n_hits)
    if i%1000==0:
        print("[{}] Average reward: [{}]".format(i, expected_return))

[0] Average reward: [0.00, 0.00, nan, nan, nan, nan]
[1000] Average reward: [0.00, 1.04, 2.09, 2.93, 3.83, 5.00]
[2000] Average reward: [0.00, 0.96, 1.90, 2.77, 3.76, 5.00]
[3000] Average reward: [0.00, 0.95, 1.87, 2.80, 3.82, 5.00]
[4000] Average reward: [0.00, 0.95, 1.90, 2.84, 3.88, 5.00]
[5000] Average reward: [0.00, 1.00, 1.98, 2.90, 3.89, 5.00]
[6000] Average reward: [0.00, 1.01, 2.00, 2.95, 3.93, 5.00]
[7000] Average reward: [0.00, 1.00, 1.99, 2.95, 3.95, 5.00]
[8000] Average reward: [0.00, 1.01, 2.00, 2.95, 3.96, 5.00]
[9000] Average reward: [0.00, 1.00, 1.99, 2.94, 3.94, 5.00]
[10000] Average reward: [0.00, 0.99, 1.97, 2.92, 3.92, 5.00]


<ipython-input-4-b19b0ba98f43>:31: RuntimeWarning: invalid value encountered in true_divide
  expected_return = ', '.join(f'{q:.2f}' for q in value_sum / n_hits)
